In [7]:
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib widget


import numpy as np
from astropy.io import fits as fitsio
from scipy.signal import find_peaks, savgol_filter
from scipy.optimize import curve_fit
from scipy import interpolate

from openhsi.utils import *

hdulist = fitsio.open("../openhsi/test_files/arc.fits")

arcimg = np.rot90(hdulist[0].data, -1)
arcimg = arcimg * 1.0 / np.max(arcimg, axis=1)[:, None]

spatialaxis=np.arange(0,arcimg.shape[0])
waveaxis=np.arange(0,arcimg.shape[1])

In [8]:
plt.figure().clf()
imgplot = plt.imshow(np.log10(arcimg.transpose()),origin='lower')
plt.ylabel('Detector Pixel (Wavelength Axis)')
plt.xlabel('Detector Pixels (Spatial Axis)')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
spatial_col_skip=1
fits = np.zeros((len(range(1, arcimg.shape[0], spatial_col_skip))+1, 28))

spec = arcimg[0, :]
peaks, properties = find_peaks(spec, height=0.01, width=1.5, prominence=0.01)


y0 = np.zeros((peaks.size * 3))
y0[0::3] = peaks
y0[1::3] = properties["peak_heights"]
y0[2::3] = properties["widths"] * 0.5
y0 = np.append(y0, 0.02)

fits[0,:], pcov = curve_fit(NGaussFunc, waveaxis, spec, p0=y0)
i=0
for col in tqdm(range(1, arcimg.shape[0], spatial_col_skip)):
    i+=1
    spec = arcimg[col, :]
    y0 = fits[i-1,:]
    fits[i,:], pcov = curve_fit(func, waveaxis, spec, p0=y0)

100%|██████████| 895/895 [00:22<00:00, 40.14it/s]


In [14]:
plt.figure()

pos=fits[:,range(0,fits.shape[1]-1,3)]
peakheight=fits[:,range(0,fits.shape[1]-1,3)]
width=fits[:,range(2,fits.shape[1]-1,3)]

smooth_pos = np.zeros_like(pos)

for row in tqdm(range(0, pos.shape[1])):
    smooth_pos[:,row] = savgol_filter(pos[:,row], 21, 3)
    
plt.subplot(211)
plt.plot(spatialaxis[range(0, arcimg.shape[0], spatial_col_skip)], smooth_pos - np.mean(pos,axis=0)) 
plt.plot(spatialaxis[range(0, arcimg.shape[0], spatial_col_skip)], pos - np.mean(pos,axis=0))
plt.title('Spectral Line Positions (Smoothed and Fit)')
plt.ylabel('Detector Pixel (Wavelength Axis; Centred on Mean of Each Line)')
plt.xlabel('Detector Pixels (Spatial Axis)')

plt.subplot(212)
plt.plot(spatialaxis[range(0, arcimg.shape[0], spatial_col_skip)], smooth_pos - pos) #-np.mean(fits[:,range(0,fits.shape[1]-1,3)],axis=0) #- np.mean(pos,axis=0)
plt.title('Difference of Smoothed and Fit Position')
plt.ylabel('Detector Pixel (Wavelength Axis)')
plt.xlabel('Detector Pixels (Spatial Axis)')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

100%|██████████| 9/9 [00:00<00:00, 297.29it/s]


In [15]:
plt.figure()
plt.subplot(211)
plt.plot(spatialaxis[range(0, arcimg.shape[0], spatial_col_skip)],pos)
plt.title('Fit Line Positon')
plt.xlabel('Detector Pixels (Spatial Axis)')
plt.ylabel('Detector Pixel (Wavelength Axis)')

plt.subplot(212)
plt.plot(spatialaxis[range(0, arcimg.shape[0], spatial_col_skip)],width*2.355)
plt.title('Fit Line Width (FWHM)')
plt.xlabel('Detector Pixels (Spatial Axis)')
plt.ylabel('Detector Pixel (Wavelength Axis)')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
spectral_lines=np.asarray([4358.328, 5460.735, 5769.598, 5790.663, 6965.4307, 7067.2175, 7272.9359, 7383.9805, 7503.8691])/10
smooth_pos.shape

wavecal=np.zeros((smooth_pos.shape[0], waveaxis.shape[0]))
p=[]
for i in range(0,smooth_pos.shape[0]):
    z = np.polyfit(smooth_pos[i,:], spectral_lines, 4)
    p.append(np.poly1d(z))
    wavecal[i,:]=p[-1](waveaxis)
    

plt.figure()
for row in range(0,896,64):
    plt.plot(wavecal[row,:],arcimg[row,:]+row/256)
    
#plt.plot(spectral_lines,p[100](smooth_pos[100,:])-spectral_lines,'x')
#minmax(p(waveaxis))

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
# min and max wave in all cols.
minwave=wavecal.min(axis=1).max()  
maxwave=wavecal.max(axis=1).min()
minwavedelta=np.diff(wavecal,axis=1).min()

newwave=np.arange(minwave, maxwave, minwavedelta)

interpimg=np.zeros((wavecal.shape[0], newwave.shape[0]))

for col in range(0,wavecal.shape[0]):
    f = interpolate.interp1d(wavecal[col,:], arcimg[col,:])
    interpimg[col,:]=f(newwave)


In [71]:
plt.figure().clf()
imgplot = plt.imshow(np.log10(interpimg.transpose()),origin='lower')
plt.ylabel('Detector Pixel (Wavelength Axis)')
plt.xlabel('Dfitsioetector Pixels (Spatial Axis)')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [70]:
fitsio.writeto("arc_cal.fits", interpimg, overwrite=True)